#Sources
- Chat Bubbles CSS
    - https://chatableapps.com/technology/coding-css-chat-bubbles-a-step-by-step-guide-to-creating-stylish-chat-interfaces/
- generators
    - https://docs.python.org/3/tutorial/classes.html
- gradio
    - https://www.machinelearningnuggets.com/gradio-tutorial/
- html
    - https://borstch.com/snippet/negotiation-simulation-game

In [ ]:
"5b324421-c65f-4fac-b7ff-fee5be1a1064"

In [19]:
import gradio as gr
import json
import time
import os
import random

# Load the file and get the simulations
def load_conversations(file_path):
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"The file {file_path} does not exist.")
    conversations = []
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            conversations.append(json.loads(line))
    return conversations

# JSONL file
negotiation_chat_file = r"C:\Users\Claire Personal\OneDrive - Saint Mary's College of California/Final Deliverables/Demo/negotiation_finetune_data_base.jsonl"
conversations = load_conversations(negotiation_chat_file)

# Set of used conversation IDs to avoid repetition
used_conversation_ids = set()
stop_simulation = False  # Flag to stop the simulation
current_conversation_id = None  # Track the current conversation being simulated

# Exclusion list: Conversations to skip
excluded_conversation_ids = {"5b324421-c65f-4fac-b7ff-fee5be1a1064"}

# Retrieve a random conversation ID that hasn't been used and isn't excluded
def get_random_unused_conversation():
    global used_conversation_ids

    # Reset used IDs if all valid conversations have been simulated
    valid_conversations = [
        convo for convo in conversations 
        if convo.get('ConversationID') not in used_conversation_ids 
        and convo.get('ConversationID') not in excluded_conversation_ids
    ]

    if not valid_conversations:
        used_conversation_ids.clear()
        print("All valid conversations have been simulated. Resetting the used list.")
        valid_conversations = [
            convo for convo in conversations 
            if convo.get('ConversationID') not in excluded_conversation_ids
        ]
    
    # Select a random conversation
    selected_conversation = random.choice(valid_conversations)
    return selected_conversation

# Run the selected negotiation/simulation of self-play
def simulate_conversation():
    global stop_simulation, current_conversation_id, used_conversation_ids
    stop_simulation = False  # Reset the stop flag when starting a new simulation
    
    selected_conversation = get_random_unused_conversation()
    current_conversation_id = selected_conversation.get('ConversationID')
    used_conversation_ids.add(current_conversation_id)  # Mark it as used immediately
    messages = selected_conversation.get('Negotiation', [])
    
    if not messages:
        yield f"<div>No messages found for Conversation ID: {current_conversation_id}</div>"
        return

    chat_html = f"<div>Simulation ID: {current_conversation_id}</div><br>"
    for msg in messages:
        if stop_simulation:
            yield gr.update(value="<div>Simulation interrupted. Click 'Start Simulation' to start a new one.</div>")
            return
        
        # Create message bubbles
        if msg['role'] == "Employee":
            chat_html += f"""
            <div style="text-align: left; margin: 15px;">
                <div style="display: inline-block; padding: 10px; border-radius: 10px; 
                            background-color: #d1f4ff; max-width: 60%; font-size: 30px;">
                    <b>{msg['role']}</b>: {msg['content']}
                </div>
            </div>
            """
        elif msg['role'] == "HR Manager":
            chat_html += f"""
            <div style="text-align: right; margin: 15px;">
                <div style="display: inline-block; padding: 10px; border-radius: 10px; 
                            background-color: #e6ffe6; max-width: 60%; font-size: 30px;">
                    <b>{msg['role']}</b>: {msg['content']}
                </div>
            </div>
            """

        # Yield the updated chat state to the UI
        yield gr.update(value=chat_html)
        time.sleep(2.5)  # Adjustable delay

    # After all messages have been displayed, show simulation has ended
    chat_html += """
    <div style="text-align: center; margin: 20px; font-size: 15px;">
        Negotiation simulation ended.
    </div>
    """
    yield gr.update(value=chat_html)

# Function to stop the simulation
def stop_current_simulation():
    """Set the stop_simulation flag to True."""
    global stop_simulation
    stop_simulation = True
    return "<div>Simulation interrupted. Click 'Start Simulation' to start a new one.</div>"

# Gradio UI
with gr.Blocks() as demo:
    gr.Markdown("## Self-Play Negotiation Simulation Demo")
    gr.Markdown("Click 'Start Simulation' to begin the simulation or 'Stop Simulation' to interrupt it.")
    
    start_button = gr.Button("Start Simulation")
    stop_button = gr.Button("Stop Simulation")
    output_box = gr.HTML(label="Live Chat Simulation")

    # Trigger live conversation simulation
    start_button.click(simulate_conversation, inputs=None, outputs=output_box)
    stop_button.click(stop_current_simulation, inputs=None, outputs=output_box)

# Launch the Gradio interface
demo.launch()


* Running on local URL:  http://127.0.0.1:7877

To create a public link, set `share=True` in `launch()`.
